In [2]:
import numpy as np
import datetime
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')

backend_sim_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))
backend_real_jakarta = provider.get_backend('ibmq_jakarta')
backend_sim = Aer.get_backend('qasm_simulator')

/var/folders/gq/pyfb45qx3wg52k7kygqcjy840000gn/T/ipykernel_5162/3644472474.py:3: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.mitigation.measurement import CompleteMeasFitter


In [14]:
qr = QuantumRegister(3)
qc = QuantumCircuit(qr)

qc.h(qr[0])
qc.cx(qr[0],qr[1])
#qc.cx(qr[1],qr[2])
qc.measure_all()

qc.draw()

┌───┐      ░ ┌─┐      
 q19_0: ┤ H ├──■───░─┤M├──────
        └───┘┌─┴─┐ ░ └╥┘┌─┐   
 q19_1: ─────┤ X ├─░──╫─┤M├───
             └───┘ ░  ║ └╥┘┌─┐
 q19_2: ───────────░──╫──╫─┤M├
                   ░  ║  ║ └╥┘
meas: 3/══════════════╩══╩══╩═
                      0  1  2

In [29]:
job = execute(qc, backend=backend_sim_jakarta, shots=80)

In [30]:
counts = job.result().get_counts()

qui ad esempio puoi vedere i counts dello stato "011".

In [31]:
counts["011"]

27

però attenzione perchè se non ne conta nemmeno uno ti da errore, e io per un po non avevo capito questa cosa e ci ho smattato. come in questo caso:

In [34]:
counts["111"]

KeyError: '111'

per risolvere questo errore basta mettere un if prima di usare il dict:

In [36]:
if "111" in counts.keys():
    print(counts["111"])
else:
    print(0)

0


quindi se devi sommare tutti gli stati com M = 1 basta fare così:

In [37]:
stati = ["100","010","001"]
numero_stati_m_1 = 0

for stato in stati:
    if stato in counts.keys():
        numero_stati_m_1 += counts[stato]

numero_stati_m_1

17

e questo è tutto.